In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

data=pd.read_csv('Crypto_historical_data.csv')
print(data)


                             Date        Open        High         Low  \
0       2017-11-09 00:00:00+00:00  308.644989  329.451996  307.056000   
1       2017-11-10 00:00:00+00:00  320.670990  324.717987  294.541992   
2       2017-11-11 00:00:00+00:00  298.585999  319.453003  298.191986   
3       2017-11-12 00:00:00+00:00  314.690002  319.153015  298.513000   
4       2017-11-13 00:00:00+00:00  307.024994  328.415009  307.024994   
...                           ...         ...         ...         ...   
350100  2025-12-05 00:00:00+00:00    0.188421    0.192632    0.178459   
350101  2025-12-06 00:00:00+00:00    0.183609    0.188021    0.180671   
350102  2025-12-07 00:00:00+00:00    0.186384    0.187851    0.177159   
350103  2025-12-08 00:00:00+00:00    0.178526    0.182188    0.176597   
350104  2025-12-10 00:00:00+00:00    0.180905    0.180905    0.176723   

             Close      Volume     ticker      name  
0       320.884003   893249984    ETH-USD  Ethereum  
1       299.252

In [ ]:
date_col = 'Date'
non_numeric_cols = ['ticker', 'name']  
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']  
numeric_data = data[numeric_cols].copy() 
non_numeric_data = data[non_numeric_cols + [date_col]].copy()  
clear_numeric = numeric_data.copy()

for i in range(len(numeric_cols)):
    col = numeric_cols[i]  
    q1 = numeric_data[col].quantile(0.25)
    q3 = numeric_data[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    mask = (numeric_data[col] >= lower_bound) & (numeric_data[col] <= upper_bound)
    clear_numeric = clear_numeric[mask]
remaining_indices = clear_numeric.index
clear_non_numeric = non_numeric_data.loc[remaining_indices].reset_index(drop=True)
clear_numeric = clear_numeric.reset_index(drop=True)
clear_combined = pd.concat([clear_non_numeric, clear_numeric], axis=1)
clear_combined = clear_combined.dropna()


X = clear_combined[numeric_cols]
iso = IsolationForest(
    contamination=0.01, 
    random_state=42,
    n_estimators=100
)
iso.fit(X)
y_pred = iso.predict(X)
df_clean = clear_combined[y_pred == 1]  
df_outliers = clear_combined[y_pred == -1]  
final_columns_order = [date_col] + numeric_cols + non_numeric_cols
df_clean = df_clean[final_columns_order]

print(df_clean.head())

Исходный размер данных: (350105, 8)


C:\Users\Денис\AppData\Local\Temp\ipykernel_4484\3089463281.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clear_numeric = clear_numeric[mask]
C:\Users\Денис\AppData\Local\Temp\ipykernel_4484\3089463281.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clear_numeric = clear_numeric[mask]
C:\Users\Денис\AppData\Local\Temp\ipykernel_4484\3089463281.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clear_numeric = clear_numeric[mask]
C:\Users\Денис\AppData\Local\Temp\ipykernel_4484\3089463281.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clear_numeric = clear_numeric[mask]


                        Date     Open     High      Low    Close    Volume  \
0  2017-11-09 00:00:00+00:00  2.05314  2.17423  1.89394  1.99077  19192200   
1  2017-11-10 00:00:00+00:00  2.00773  2.06947  1.64478  1.79684  11155000   
2  2017-11-11 00:00:00+00:00  1.78628  1.91775  1.61429  1.67047   8178150   
3  2017-11-12 00:00:00+00:00  1.66889  1.67280  1.46256  1.51969  15298700   
4  2017-11-13 00:00:00+00:00  1.52601  1.73502  1.51760  1.68662  12238800   

    ticker name  
0  BNB-USD  BNB  
1  BNB-USD  BNB  
2  BNB-USD  BNB  
3  BNB-USD  BNB  
4  BNB-USD  BNB  
